In [1]:
import sys
sys.path.append("/veRL/veRL/cyk/vla-mix/verl")
sys.path.append("/file_system/cyk/vla_mix/LIBERO/")
import functools
import multiprocessing as mp

import numpy as np
import torch


original_torch_load = torch.load


# @functools.wraps(original_torch_load)
# def new_torch_load(*args, **kwargs):
#     if "weights_only" not in kwargs:
#         kwargs["weights_only"] = False
#     return original_torch_load(*args, **kwargs)


# torch.load = new_torch_load

In [2]:
import os
import torch
import numpy as np
import json
import torch
import numpy as np
import math
from PIL import Image
from io import BytesIO
from verl.models.openvla_oft.configuration_prismatic import OpenVLAConfig
from verl.models.openvla_oft.modeling_prismatic import OpenVLAForActionPrediction
from verl.models.openvla_oft.processing_prismatic import PrismaticImageProcessor, PrismaticProcessor
# from transformers import AutoConfig, AutoImageProcessor, AutoModelForVision2Seq, AutoProcessor


Using LIBERO constants:
  NUM_ACTIONS_CHUNK = 8
  ACTION_DIM = 7
  PROPRIO_DIM = 8
  ACTION_PROPRIO_NORMALIZATION_TYPE = NormalizationType.BOUNDS_Q99
If needed, manually set the correct constants in `prismatic/vla/constants.py`!


In [3]:
local_path = "/file_system/common-models/Haozhan72-kangsheng/Openvla-oft-SFT-libero10-trajall/"
INFER_DEVICE = "cuda:0"
processor = PrismaticProcessor.from_pretrained(local_path, trust_remote_code=True)
tokenizer=processor.tokenizer

# override model kwargs
actor_model_config = OpenVLAConfig.from_pretrained(local_path, trust_remote_code=True)

torch_dtype = torch.float32
actor_module = OpenVLAForActionPrediction.from_pretrained(
                                        pretrained_model_name_or_path=local_path,
                                        torch_dtype=torch_dtype,
                                        #attn_implementation="flash_attention_2",
                                        config=actor_model_config,              
                                        trust_remote_code=True,
                                    )
actor_module.vision_backbone.set_num_images_in_input(1)
dataset_statistics_path = os.path.join(local_path, "dataset_statistics.json")
if os.path.isfile(dataset_statistics_path):
    with open(dataset_statistics_path, "r") as f:
        norm_stats = json.load(f)
    actor_module.norm_stats = norm_stats
actor_module = actor_module.to(INFER_DEVICE)
actor_module.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
PrismaticForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
OpenVLAForActionPrediction has generative capa

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

PrismaticForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
OpenVLAForActionPrediction has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit

OpenVLAForActionPrediction(
  (vision_backbone): PrismaticVisionBackbone(
    (featurizer): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp)

In [ ]:
from omegaconf import OmegaConf

from verl.envs.libero.libero_env import LiberoEnv
batch_size = 16
cfg_dict = {
        "task_suite_name": "libero_10",
        "num_envs": batch_size,
        "group_size": batch_size,
        "num_group": 1,
        "seed": 0,
        "use_fixed_reset_state_ids": False,
        "ignore_terminations": False,
        "auto_reset": True,
        "max_episode_steps": 512,
        "use_rel_reward": False,
        "reward_coef": 1.0,
        "only_eval": False,
        "use_ordered_reset_state_ids": False,
        "num_images_in_input": 2,
        "init_params": {
            "camera_depths": False,
            "camera_heights": 512,
            "camera_widths": 512,
            "camera_names": ["agentview", "robot0_eye_in_hand"],
        },
        "controller_configs": {
            "type": "OSC_POSE",
            "input_max": 1,
            "input_min": -1,
            "output_max": [0.05, 0.05, 0.05, 0.5, 0.5, 0.5],
            "output_min": [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5],
            "kp": 150,
            "damping_ratio": 1,
            "impedance_mode": "fixed",
            "kp_limits": [0, 300],
            "damping_ratio_limits": [0, 10],
            "position_limits": None,
            "orientation_limits": None,
            "uncouple_pos_ori": True,
            "control_delta": True,
            "interpolation": None,
            "ramp_ratio": 0.2,
        },
        "video_cfg": {
            "save_video": True,
            "video_base_dir": "/tmp/videos",
        },
    }
cfg = OmegaConf.create(cfg_dict)

: 

In [ ]:
from collections import defaultdict
import copy
import torch.cuda.profiler as profiler
from tqdm import trange



# batch_size = 32
# batch_size = 8
# max_steps = 200
max_steps = 512
# max_steps = 1024
# max_steps = 256
action_chunks_len = 8
libero_env = LiberoEnv(cfg, rank=0, world_size=1)
libero_env.is_start = True
# The first call to step with actions=None will reset the environment
obs_venv, _, terminated_venv, truncated_venv, info_venv = libero_env.step(actions=None)
print("Initial data:", obs_venv)
# print("Initial data:", init_data)
# task_descriptions = [init_data['task_description']]
# print(f"Task description: {task_descriptions}")

# valid_video[init_data['task_file_name']].extend(init_data['valid_images'])
# env_data = copy.deepcopy(init_data)
# env_obs = env_data['obs']
# for step in trange(max_steps // action_chunks_len):
#     # print("Step:", step)
#     # prof.step()
#     inputs = [_obs_to_input(env_obs)]
#     vla_input = process_input(inputs, task_descriptions, config)
#     # vla_input.update(meta_info)
#     vla_output = _generate_one_step(vla_input)
#     actions = vla_output["action"]
#     step_data = {
#         "responses": vla_output["responses"],
#         "input_ids": vla_output["input_ids"],
#         "attention_mask": vla_output["attention_mask"],
#         "pixel_values": vla_output["pixel_values"],
#         "action": actions,
#         "step": step
#     }
#     vla_history.append(step_data)
    

#     result = libero_env.step(actions[0])
#     valid_video[init_data['task_file_name']].extend(result['valid_images'])
#     env_obs = result["obs"]

#     step += action_chunks_len
#     complete = result["complete"]
#     if complete:
#         print(f"Task completed at step {step}.")
#         break
    

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
get task and trial id [ 50 100 150 200 250 300 350 400 450 500] [425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425
 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425
 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425 425
 425 425 425 425 425 425 425 425 425 425] [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8] [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

get task and trial id [ 50 100 150 200 250 300 350 400 450 500] [318 255 134 153  20  37   8  87 406 324 456 251 303 485 364 316 271 279
 467 138 407 335   1 197 428 277  16 382 364 423  87  44 431  11 270  40
 149 240 211 201  14   2  62   4 335 262 323 128 307 382 191 230 498 402
 490 189 342 475 325 420 344 352 194 437] [6, 5, 2, 3, 0, 0, 0, 1, 8, 6, 9, 5, 6, 9, 7, 6, 5, 5, 9, 2, 8, 6, 0, 3, 8, 5, 0, 7, 7, 8, 1, 0, 8, 0, 5, 0, 2, 4, 4, 4, 0, 0, 1, 0, 6, 5, 6, 2, 6, 7, 3, 4, 9, 8, 9, 3, 6, 9, 6, 8, 6, 7, 3, 8] [18, 5, 34, 3, 20, 37, 8, 37, 6, 24, 6, 1, 3, 35, 14, 16, 21, 29, 17, 38, 7, 35, 1, 47, 28, 27, 16, 32, 14, 23, 37, 44, 31, 11, 20, 40, 49, 40, 11, 1, 14, 2, 12, 4, 35, 12, 23, 28, 7, 32, 41, 30, 48, 2, 40, 39, 42, 25, 25, 20, 44, 2, 44, 37]


EOFError: 

: 

: 

In [ ]:
obs_venv

{'images_and_states': {'full_image': tensor([[[[ 13,  13,  13],
            [ 13,  13,  13],
            [ 13,  13,  13],
            ...,
            [117, 115, 112],
            [117, 115, 111],
            [116, 114, 110]],
  
           [[ 13,  13,  13],
            [ 13,  13,  13],
            [ 13,  13,  13],
            ...,
            [117, 116, 112],
            [116, 114, 110],
            [116, 114, 110]],
  
           [[ 13,  13,  13],
            [ 13,  13,  13],
            [ 13,  13,  13],
            ...,
            [118, 116, 112],
            [116, 114, 110],
            [117, 114, 111]],
  
           ...,
  
           [[ 79,  57,  45],
            [ 76,  55,  44],
            [ 73,  54,  44],
            ...,
            [ 83,  57,  45],
            [ 84,  58,  46],
            [ 85,  59,  47]],
  
           [[ 77,  56,  46],
            [ 76,  55,  46],
            [ 74,  54,  45],
            ...,
            [ 82,  57,  44],
            [ 82,  58,  45],
    

: 

: 

In [ ]:
from PIL import Image
import torchvision.transforms.functional as F
from torch.nn.utils.rnn import pad_sequence
def center_crop_image(image: Image.Image) -> Image.Image:

    crop_scale = 0.9
    orig_w, orig_h = image.size
    image_tensor = F.to_tensor(image)
    crop_h = int(orig_h * crop_scale)
    crop_w = int(orig_w * crop_scale)
    image_tensor = F.center_crop(image_tensor, (crop_h, crop_w))
    image_tensor = F.resize(image_tensor, (orig_h, orig_w))
    final_image = F.to_pil_image(image_tensor)
    
    final_image = final_image.convert("RGB")
    return final_image

# def process_input(inputs:list, task_descriptions:list, config):
    
#     batchdata = {"input_ids":[],"attention_mask":[],"pixel_values":[]}  
    
#     for i in range(len(inputs)):
#         input = inputs[i]
#         task_description = task_descriptions[i]
        
#         image = Image.fromarray(input["full_image"]).convert("RGB")
#         if config["center_crop"]:
#             image = center_crop_image(image)
#         prompt = f"In: What action should the robot take to {task_description.lower()}?\nOut:"
#         batch_feature  = processor(prompt, image)
        
#         if "wrist_image" in input.keys():
#             wrist_image = Image.fromarray(input["wrist_image"]).convert("RGB")
#             if config["center_crop"]:
#                 wrist_image = center_crop_image(wrist_image)
#             wrist_batch_feature = processor(prompt, wrist_image)
#             primary_pixel_values = batch_feature["pixel_values"]
#             batch_feature["pixel_values"] = torch.cat([primary_pixel_values] + [wrist_batch_feature["pixel_values"]], dim=1)
            
#         input_ids = batch_feature["input_ids"]
#         attention_mask = batch_feature["attention_mask"]
#         pixel_values = batch_feature["pixel_values"]
        
#         if not torch.all(input_ids[:, -1] == 29871):
#             input_ids = torch.cat(
#                 (input_ids, torch.unsqueeze(torch.Tensor([29871]).long(), dim=0).to(input_ids.device)), dim=1
#             )
#             attention_mask = torch.cat(
#                 (attention_mask, torch.unsqueeze(torch.Tensor([True]).bool(), dim=0).to(attention_mask.device)), dim=1
#             )
        
#         batchdata["input_ids"].append(input_ids)    
#         batchdata["attention_mask"].append(attention_mask)    
#         batchdata["pixel_values"].append(pixel_values)    
    
    
#     device = torch.device('cuda') 
    
#     batchdata["input_ids"] = [x.transpose(0, 1) for x in batchdata["input_ids"]]
#     batchdata["attention_mask"] = [x.transpose(0, 1) for x in batchdata["attention_mask"]]
#     batchdata["input_ids"] = pad_sequence(batchdata["input_ids"], batch_first=True, padding_value=processor.tokenizer.pad_token_id).squeeze(-1).to(device)
#     batchdata["attention_mask"] = pad_sequence(batchdata["attention_mask"], batch_first=True, padding_value=0).squeeze(-1).to(device)
    
#     padding_mask = batchdata["input_ids"].ne(processor.tokenizer.pad_token_id)
#     assert  torch.all(padding_mask==batchdata["attention_mask"].ne(0))
#     padding_mask = ~padding_mask
#     padding_mask = padding_mask.int() 
#     sorted_indices = torch.argsort(padding_mask, dim=1, descending=True, stable=True)
#     batchdata["input_ids"] = torch.gather(batchdata["input_ids"], 1, sorted_indices)
#     batchdata["attention_mask"] = torch.gather(batchdata["attention_mask"], 1, sorted_indices)
    
    
#     batchdata["pixel_values"] = torch.cat(batchdata["pixel_values"] , dim=0).to(device)
#     assert torch.all(batchdata["attention_mask"].ne(0) == batchdata["input_ids"].ne(processor.tokenizer.pad_token_id))

#     return batchdata
def process_input(venv_obs, config):
    
    batchdata = {"input_ids":[],"attention_mask":[],"pixel_values":[]}  
    
    task_descriptions = venv_obs["task_descriptions"]
    images_and_states = venv_obs["images_and_states"]
    for i in range(len(venv_obs["task_descriptions"])):
        task_description = task_descriptions[i]
        
        image = Image.fromarray(images_and_states["full_image"][i].numpy()).convert("RGB")
        if config["center_crop"]:
            image = center_crop_image(image)
        prompt = f"In: What action should the robot take to {task_description.lower()}?\nOut:"
        batch_feature  = processor(prompt, image)
        
        # if "wrist_image" in venv_obs.keys():
        #     wrist_image = Image.fromarray(input["wrist_image"]).convert("RGB")
        #     if config["center_crop"]:
        #         wrist_image = center_crop_image(wrist_image)
        #     wrist_batch_feature = processor(prompt, wrist_image)
        #     primary_pixel_values = batch_feature["pixel_values"]
        #     batch_feature["pixel_values"] = torch.cat([primary_pixel_values] + [wrist_batch_feature["pixel_values"]], dim=1)
            
        input_ids = batch_feature["input_ids"]
        attention_mask = batch_feature["attention_mask"]
        pixel_values = batch_feature["pixel_values"]
        
        if not torch.all(input_ids[:, -1] == 29871):
            input_ids = torch.cat(
                (input_ids, torch.unsqueeze(torch.Tensor([29871]).long(), dim=0).to(input_ids.device)), dim=1
            )
            attention_mask = torch.cat(
                (attention_mask, torch.unsqueeze(torch.Tensor([True]).bool(), dim=0).to(attention_mask.device)), dim=1
            )
        
        batchdata["input_ids"].append(input_ids)    
        batchdata["attention_mask"].append(attention_mask)    
        batchdata["pixel_values"].append(pixel_values)    
    
    
    device = torch.device('cuda') 
    
    batchdata["input_ids"] = [x.transpose(0, 1) for x in batchdata["input_ids"]]
    batchdata["attention_mask"] = [x.transpose(0, 1) for x in batchdata["attention_mask"]]
    batchdata["input_ids"] = pad_sequence(batchdata["input_ids"], batch_first=True, padding_value=processor.tokenizer.pad_token_id).squeeze(-1).to(device)
    batchdata["attention_mask"] = pad_sequence(batchdata["attention_mask"], batch_first=True, padding_value=0).squeeze(-1).to(device)
    
    padding_mask = batchdata["input_ids"].ne(processor.tokenizer.pad_token_id)
    assert  torch.all(padding_mask==batchdata["attention_mask"].ne(0))
    padding_mask = ~padding_mask
    padding_mask = padding_mask.int() 
    sorted_indices = torch.argsort(padding_mask, dim=1, descending=True, stable=True)
    batchdata["input_ids"] = torch.gather(batchdata["input_ids"], 1, sorted_indices)
    batchdata["attention_mask"] = torch.gather(batchdata["attention_mask"], 1, sorted_indices)
    
    
    batchdata["pixel_values"] = torch.cat(batchdata["pixel_values"] , dim=0).to(device)
    assert torch.all(batchdata["attention_mask"].ne(0) == batchdata["input_ids"].ne(processor.tokenizer.pad_token_id))

    return batchdata

: 

: 

In [ ]:
# DO_SAMPLE = True
DO_SAMPLE = False
TEMP = 1.6
# TEMP = 0.2
UNNORM_KEY = "libero_10"
UNNORM_KEY = f"{UNNORM_KEY}_no_noops"
MAX_PROMPT_LENGTH = 512
def pad_sequence_to_length(tensors, max_seq_len, pad_token_id, left_pad=False):
    """
    pad a 2D tensors (e.g. responses, logprobs) in the last dim to max_seq_length.
    input shape: [bs, seq_length]
    output shape: [bs, max_seq_length]
    (0, max_seq_len - tensors.shape[-1]) means right pad to max_seq_length and no left pad
    """
    if tensors.shape[-1] >= max_seq_len:
        return tensors
    pad_tuple = (max_seq_len - tensors.shape[-1], 0) if left_pad else (0, max_seq_len - tensors.shape[-1])
    return torch.nn.functional.pad(tensors, pad_tuple, 'constant', pad_token_id)

@torch.no_grad()
def _generate_one_step(prompts: dict):
    idx = prompts['input_ids']  # (bs, prompt_length)
    attention_mask = prompts['attention_mask']  # left-padded attention_mask
    pixel_values = prompts["pixel_values"]



    # make sampling args can be overriden by inputs
    do_sample = prompts.get('do_sample', DO_SAMPLE)


    temperature = prompts.get('temperature', TEMP)

    #generation_config = GenerationConfig(temperature=temperature, top_p=top_p, top_k=top_k)

    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        actions, response = actor_module.generate_action_verl(
            input_ids=idx,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            padding_idx = processor.tokenizer.pad_token_id,
            do_sample=do_sample,
            unnorm_key=UNNORM_KEY,
            temperature=temperature,)
    
    
    assert processor.tokenizer.pad_token_id is not None

    assert idx.ndim == 2
    idx = pad_sequence_to_length(idx,max_seq_len=MAX_PROMPT_LENGTH,pad_token_id=processor.tokenizer.pad_token_id,left_pad=True)
    
    assert attention_mask.ndim == 2
    attention_mask = pad_sequence_to_length(attention_mask,max_seq_len=MAX_PROMPT_LENGTH,pad_token_id=0,left_pad=True)
    
    
    assert idx.device.type == 'cuda'
    assert response.device.type == 'cuda'
    #assert seq.device.type == 'cuda'
    assert attention_mask.device.type == 'cuda'
    assert pixel_values.device.type == 'cuda'
    batch ={
            'responses': response,
            'input_ids': idx,
            'attention_mask': attention_mask,
            "pixel_values":pixel_values,
            "action":actions,
        }

    return batch

: 

: 

In [ ]:
task_descriptions = obs_venv['task_descriptions']
print(f"Task description: {task_descriptions}")
config = {
    "center_crop": True,
    "num_steps_wait": 10
}
vla_history = []
for step in trange(max_steps // action_chunks_len):

    # print("Step:", step)
    # prof.step()
    vla_input = process_input(obs_venv, config)
    # vla_input.update(meta_info)
    vla_output = _generate_one_step(vla_input)
    actions = vla_output["action"]
    if step < 1:
        actions = torch.tensor([[0, 0, 0, 0, 0, 0, 1]]).repeat(batch_size, action_chunks_len, 1).numpy()
        print(actions.shape)    
    step_data = {
        "responses": vla_output["responses"],
        "input_ids": vla_output["input_ids"],
        "attention_mask": vla_output["attention_mask"],
        "pixel_values": vla_output["pixel_values"],
        "action": actions,
        "step": step
    }
    vla_history.append(step_data)

    # TODO(caiyunke.astra): check if need to invert the gripper action
    # Invert the gripper action
    actions[..., -1] =actions[..., -1] *  -1.0
    

    # result = libero_env.step(actions[0])
    result = libero_env.chunk_step(actions)
    obs_venv, _, terminated_venv, truncated_venv, info_venv = result
    step += action_chunks_len
    if terminated_venv.any() or truncated_venv.any():
        print(f"Step {step}:")
        if terminated_venv.any():
            print(f"  Terminated: {terminated_venv.cpu().numpy()}")
        if truncated_venv.any():
            print(f"  Truncated: {truncated_venv.cpu().numpy()}")
        if info_venv:
            print(f"  Info: {info_venv}")
    # if complete:
    #     print(f"Task completed at step {step}.")
    #     break
success_count = info_venv["final_info"]["episode"]["success_once"].int().sum().item()
print(f"  success rate: {success_count} / {batch_size}")

Task description: ['put the white mug on the plate and put the chocolate pudding to the right of the plate', 'pick up the book and place it in the back compartment of the caddy', 'turn on the stove and put the moka pot on it', 'put the black bowl in the bottom drawer of the cabinet and close it', 'put both the alphabet soup and the tomato sauce in the basket', 'put both the alphabet soup and the tomato sauce in the basket', 'put both the alphabet soup and the tomato sauce in the basket', 'put both the cream cheese box and the butter in the basket']


  0%|          | 0/64 [00:00<?, ?it/s]

(8, 8, 7)


  2%|▏         | 1/64 [00:02<02:42,  2.58s/it]

(8, 8, 7)


...

get task and trial id [ 50 100 150 200 250 300 350 400 450 500] [406 324 456 251 303 485 364 316] [8, 6, 9, 5, 6, 9, 7, 6] [6, 24, 6, 1, 3, 35, 14, 16]


100%|██████████| 64/64 [02:25<00:00,  2.27s/it]

Step 71:
  Truncated: [[False False False False False False False  True]
 [False False False False False False False  True]
 [False False False False False False False  True]
 [False False False False False False False  True]
 [False False False False False False False  True]
 [False False False False False False False  True]
 [False False False False False False False  True]
 [False False False False False False False  True]]
  Info: {'final_observation': {'images_and_states': {'full_image': tensor([[[[ 13,  13,  13],
          [ 13,  13,  13],
          [ 13,  13,  13],
          ...,
          [117, 115, 112],
          [117, 115, 111],
          [116, 114, 110]],

         [[ 13,  13,  13],
          [ 13,  13,  13],
          [ 13,  13,  13],
          ...,
          [117, 116, 112],
          [116, 114, 110],
          [116, 114, 110]],

         [[ 13,  13,  13],
          [ 13,  13,  13],
          [ 13,  13,  13],
          ...,
          [118, 116, 112],
          [116, 114, 

: 

: 

In [ ]:
libero_env.flush_video(video_sub_dir=f"episode_{7}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


: 

: 

In [ ]:
libero_env.env.close()

: 

: 